In [1]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import typing as tp

import numpy as np
import pandas as pd
from lightfm import LightFM
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import LightFMWrapperModel

# As first level train the model from HW4


In [4]:
users = pd.read_csv("../data/users.csv")
items = pd.read_csv("../data/items.csv")
interactions = pd.read_csv("../data/interactions.csv")

In [5]:
Columns.Datetime = "last_watch_dt"
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format="%Y-%m-%d")
max_date = interactions[Columns.Datetime].max()
interactions[Columns.Weight] = np.where(interactions["watched_pct"] > 10, 3, 1)

In [6]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

train.drop(train.query("total_dur < 300").index, inplace=True)

cold_users = set(test[Columns.User]) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

TEST_USERS = test[Columns.User].unique()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (3832711, 6)
test: (333026, 6)


In [9]:
def get_user_features(users: pd.DataFrame, interactions: pd.DataFrame, features: tp.List[str]):
    users = users.loc[users[Columns.User].isin(interactions[Columns.User])].copy()
    user_features_frames = []
    for feature in features:
        feature_frame = users.reindex(columns=[Columns.User, feature])
        feature_frame.columns = ["id", "value"]
        feature_frame["feature"] = feature
        user_features_frames.append(feature_frame)
    user_features = pd.concat(user_features_frames)
    return user_features


def get_item_features(items: pd.DataFrame, interactions: pd.DataFrame):
    items = items.loc[items[Columns.Item].isin(interactions[Columns.Item])].copy()
    items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
    genre_feature = items[["item_id", "genre"]].explode("genre")
    genre_feature.columns = ["id", "value"]
    genre_feature["feature"] = "genre"
    content_feature = items.reindex(columns=[Columns.Item, "content_type"])
    content_feature.columns = ["id", "value"]
    content_feature["feature"] = "content_type"
    item_features = pd.concat((genre_feature, content_feature))
    return item_features

In [10]:
user_features = get_user_features(users, train, ["sex", "age", "income"])
item_features = get_item_features(items, train)

In [11]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

In [12]:
max_date = interactions[Columns.Datetime].max()
min_date = interactions[Columns.Datetime].min()

print(f"min дата в interactions: {min_date}")
print(f"max дата в interactions: {max_date}")
print(f"Продолжительность: {max_date - min_date}")

ranker_days_count = 30

interactions = interactions[(interactions[Columns.Datetime] < max_date - pd.Timedelta(days=ranker_days_count))]

min дата в interactions: 2021-03-13 00:00:00
max дата в interactions: 2021-08-22 00:00:00
Продолжительность: 162 days 00:00:00


In [13]:
user_features = get_user_features(users, interactions, ["sex", "age", "income"])
item_features = get_item_features(items, interactions)

In [14]:
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

# Train the LightFM Model

In [15]:
model = LightFMWrapperModel(
    LightFM(
        no_components=8,
        loss="warp",
        random_state=42,
        learning_rate=0.05,
        user_alpha=0.3,
        item_alpha=0.2,
    ),
    epochs=1,
    num_threads=2,
)
model.fit(dataset)

# Prediction of candidates

In [17]:
top_N = 100
candidates = model.recommend(dataset.user_id_map.external_ids, dataset, top_N, True)
candidates = candidates.rename({"rank": "lfm_rank", "score": "lfm_score"}, axis=1)
candidates.head()

,user_id,item_id,lfm_score,lfm_rank
0,176549,12365,0.001082,1
1,176549,2150,0.001069,2
2,176549,13865,0.000992,3
3,176549,12138,0.000988,4
4,176549,10440,0.000962,5


In [19]:
candidates.to_csv("../data/candidates_lfm.csv", index=False)